In [127]:
from urllib.request import urlopen
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import urllib.request
import wget
import csv
from pulp import *
import operator

In [128]:
ROTOGRINDER = True
SPORTSLINE = False

In [129]:
def check_last_page():
    next_page_button = driver.find_elements_by_id("table_all_players_search_paginate")[0]
    next_page_button = next_page_button.find_elements_by_tag_name("a")[1]
    return (str(next_page_button.get_attribute('class')) != "paginate_button next")

def fix_name(name):
    name = name.lower().strip()
    name_ = name.split(" ")
    if SPORTSLINE:
        if len(name_) == 1 and name != "nene":
            return name.lower()
        if name_[-1].lower() == "edrice":
            name_[-1] = "bam"
        if name == "capela clint":
            return "clint n`dumba-capela"
        if name == "nene":
            return "nene hilario"
    if name == "porter jr otto":
        return "otto porter"
    if name == "nance jr larry":
        return "larry nance"
    if name == "luc mbah a moute":
        return "luc richard mbah a moute"
    if name_[-1].lower() == "jj":
            name_[-1] = "j.j."
    if name_[-1].lower() == "cj":
        name_[-1] = "c.j."
    if name_[-1].lower() == "tj":
        name_[-1] = "t.j."
    if name_[-1].lower() == "pj":
        name_[-1] = "p.j."
    if name_[-1].lower() == "malcom":
        name_[-1] = "malcolm"
    if len(name_) > 1:
        name = name_[-1].lower() + " "
        for i in range(0,len(name_) -1):
            name += name_[i].lower()
            if i < len(name_) -2:
                name += " "
    return name

def fix_sisal_name_teams(teams_string):
    teams_string_list = teams_string.lower().split("-")
    teams_string_list[0] = teams_string_list[0].strip()
    teams_string_list[1] = teams_string_list[1].strip()
    return (teams_string_list[0],teams_string_list[1])
        

In [130]:
##Adding adblock to chromedriver and starting it##
options = webdriver.ChromeOptions()
options.add_extension("/Users/silvio/Desktop/dunkest/adblock.crx")
driver = webdriver.Chrome(chrome_options = options)

In [131]:
##Login to dunkest##
driver.get('https://nba.dunkest.com/it/home')

before = driver.window_handles[0]
after = driver.window_handles[1]
driver.switch_to_window(after)
time.sleep(1.5)
driver.close()
time.sleep(1.5)
driver.switch_to_window(before)


username = driver.find_element_by_id("email")
password = driver.find_element_by_id("password")
username.send_keys("silvio.pavanetto@gmail.com")
password.send_keys("Salmousb1!")

#username.send_keys("kuhetave@web2mailco.com")
#password.send_keys("alpine")

driver.find_element_by_name("submit").click()

time.sleep(2)

driver.get('https://nba.dunkest.com/it/mercato-fanta-squadra-nba')

In [132]:
#Scraping info from dunkest :)
credits_total = driver.find_element_by_id("team_quotation")
credits_total = float(credits_total.text)

players = {}
coaches = {}

xpath = "//table[@id='table_all_players_search']/tbody"

while (check_last_page() == False):
    table_id = driver.find_elements_by_xpath(xpath)
    rows = table_id[0].find_elements_by_tag_name("tr") # get all of the rows in the table
    for row in rows:
        # Get the columns (all the column 2)
        name_player = str(row.find_elements_by_tag_name("td")[0].find_elements_by_class_name("player-name")[0].find_elements_by_tag_name("a")[0].get_attribute('title'))
        name_player = fix_name(name_player)
        position = str(row.find_elements_by_tag_name("td")[1].find_elements_by_class_name("player-role")[0].text)
        name_team = str(row.find_elements_by_tag_name("td")[2].find_elements_by_tag_name("a")[0].get_attribute('title'))
        credits_player = float(row.find_elements_by_tag_name("td")[4].get_attribute('data-order'))
        if position == "HC":
            coaches[name_player] = [name_team.lower(),credits_player,0.0]
        else:
            players[name_player] = [name_team.lower(),position,credits_player,0.0,0.0]

    driver.find_elements_by_id("table_all_players_search_paginate")[0].find_elements_by_tag_name("a")[1].click()

#print(players)


In [133]:
##scraping games to be played##
games = []
games_list = []
xpath_games = "//table[@class='round-list']/tbody"
table_id = driver.find_elements_by_xpath(xpath_games)
rows = table_id[0].find_elements_by_tag_name("tr") # get all of the rows in the table
for row in rows:
    home_team = str(row.find_elements_by_tag_name("td")[1].find_elements_by_tag_name("a")[0].text).lower()
    away_team = str(row.find_elements_by_tag_name("td")[3].find_elements_by_tag_name("a")[0].text).lower()
    
    #here are all the matches
    games.append([home_team,away_team]) 
    
    #here are all the teams together
    games_list.append(home_team) 
    games_list.append(away_team)

In [134]:
##SCRAPING BETS FOR THE GAMES TO BE PLAYED##
driver.execute_script("window.open('');")
time.sleep(1)
driver.switch_to.window(driver.window_handles[1])
driver.get('https://www.sisal.it/scommesse/basket/usa/nba')
time.sleep(1.5)
teams_and_bets_from_sisal = {}
xpath_games = "//div[@class='events']/div[@class='partitasingola'][@style='display: block;']/div"
xpath_teams = ".//div/p[@class='name']/a"
xpath_bets = ".//div[starts-with(@class, 'quota-label')]"
events_deep = driver.find_elements_by_xpath(xpath_games)
i = 0
for game in events_deep:
    if i % 2 == 0:
        string_teams = game.find_elements_by_xpath(xpath_teams)[0].text
        string_teams = fix_sisal_name_teams(string_teams)
        string_bets = game.find_elements_by_xpath(xpath_bets)
        teams_and_bets_from_sisal[str(string_teams[0])] = float(string_bets[0].text)
        teams_and_bets_from_sisal[str(string_teams[1])] = float(string_bets[1].text)
    i += 1

In [135]:
##download projections and connect them to dunkest data##
sportsline_url = "https://www.sportsline.com/sportsline-web/service/v1/playerProjectionsCsv?league=nba&position=all-players&sourceType=FD&page=PS&desc=false&optimal=false"
rotogrinder_url = "https://rotogrinders.com/projected-stats/nba-player.csv?site=fanduel"

sportsline = wget.download(sportsline_url)
rotogrinder = wget.download(rotogrinder_url)

In [136]:
##ALL THE DATA HAVE BEEN COLLECTED, NOW JUST LOGIC##

In [137]:
players_filtered = {} 
coaches_filtered = {}

for key, value in players.items():
    if value[0] in games_list:
        players_filtered[key] = players[key]
        
for key, value in coaches.items():
    if value[0] in games_list:
        coaches_filtered[key] = coaches[key]

In [138]:
#print(players_filtered)

In [139]:
##Function for substitute
players_filtered["luc richard mbah a moute"] = players_filtered.pop("luc mbah a moute")

KeyError: 'luc mbah a moute'

In [140]:
players_only = []
predictions = []
costs = []
roles = []
predicted_fantasy_pts = []


'''
##SPORTSLINE##
with open("/Users/silvio/Desktop/dunkest/projections.csv", 'r') as f:
    reader = csv.reader(f, delimiter=',')

    headers = next(reader)

    #writer.writerow(headers)
    for row in reader:
        
        ftm_percent = float(str(row[10])[:-1])/100
        proj = float(row[3]) + (float(row[5])*1.5) + (float(row[11])*1.5) + (float(row[12])*1.25) + float(row[13]) + (float(row[14])*1.5) - (float(row[15])*1.5) - (float(row[9])/ftm_percent - float(row[9]))
        proj = '%.3f'%proj
        players_filtered[str(row[0]).lower()][3] = proj

for k,v in players_filtered.items():
    coeff = float(v[3])/float(v[2])
    coeff = '%.4f'%coeff
    players_filtered[k] = [v[0],v[1],v[2],v[3],coeff]
'''

##ROTOGRINDER##
with open("/Users/silvio/Desktop/dunkest/nba-player.csv", 'r') as f:
    reader = csv.reader(f, delimiter=',')

    for row in reader:
        low = row[6].strip()
        high = row[5].strip()
        if len(low) > 0:
            score = (float(low)/float(high))*float(row[7].strip())
            #score = '%.4f'%score
            players_filtered[str(row[0]).lower().strip()][3] = score
            players_filtered[str(row[0]).lower().strip()][4] = float(row[7])
        else:
            players_filtered[str(row[0]).lower().strip()][3] = float(row[7])

for k,v in players_filtered.items():
    players_only.append(k)
    predictions.append(v[3])
    costs.append(v[2])
    roles.append(v[1])
    predicted_fantasy_pts.append(float(v[4]))
    

In [141]:
players_filtered_by_score = sorted(players_filtered.items(), key=lambda e: e[1][3], reverse = True)

In [142]:
##FIRST OF ALL, SELECT THE BEST COACH##

for k,v in coaches_filtered.items():
    score = 1/(teams_and_bets_from_sisal[v[0]]*float(v[1]))
    coaches_filtered[k] = [v[0],v[1],score]

coaches_sorted = sorted(coaches_filtered.items(), key=lambda e: e[1][2], reverse = True)

coach_selected = coaches_sorted[0]

In [143]:
coach_enabled = True

In [ ]:
#print(players_only,roles,costs,predictions,predicted_fantasy_pts)
credits_total = 105

In [144]:
keep_players = []
keep_players = [str(x).lower().strip() for x in input("Insert a list of players you want to keep, separated by a comma: ").split(",")]
for i in range(0,len(keep_players)):
    if (len(keep_players[i]) == 0) or (keep_players[i] not in players_filtered):
        keep_players.pop(i)
        
total_cost = 0
total_points = 0
credits_total = credits_total - sum(players_filtered[elem][2] for elem in keep_players)

for i in range(0,len(keep_players)):
    index = players_only.index(keep_players[i])
    players_only.pop(index)
    roles.pop(index)
    total_cost += costs.pop(index)
    predictions.pop(index)
    total_points += predicted_fantasy_pts.pop(index)

Insert a list of players you want to keep, separated by a comma: 


In [145]:
P = range(len(players_only))

# Declare problem instance, maximization problem
prob = LpProblem("Portfolio", LpMaximize)

# Declare decision variable x, which is 1 if a
# player is part of the portfolio and 0 else
x = LpVariable.matrix("x", list(P), 0, 1, LpInteger)

# Objective function -> Maximize votes
prob += sum(predictions[p] * x[p] for p in P)

# Constraint definition
prob += sum(x[p] for p in P) == 10
if coach_enabled:
    prob += sum(costs[p] * x[p] for p in P) <= (credits_total - coach_selected[1][1])
else:
    prob += sum(costs[p] * x[p] for p in P) <= credits_total - 7.9
prob += sum(x[p] for p in P if roles[p] == "G") == 4
prob += sum(x[p] for p in P if roles[p] == "A") == 4
prob += sum(x[p] for p in P if roles[p] == "C") == 2
#prob += x[p] for p in P if predicted_fantasy_pts[p] == max(predicted_fantasy_pts)


# Start solving the problem instance
prob.solve()

# Extract solution
portfolio = [(players_only[p],roles[p],costs[p],predictions[p],predicted_fantasy_pts[p]) for p in P if x[p].varValue]
portfolio = sorted(portfolio, key=lambda x: x[2], reverse = True)

In [146]:
##FINAL PRINT OF THE TEAM, MAXIMIZING THE TOTAL SCORE##
if coach_enabled:
    print("COACH: ")
    print("NAME: " + str(coach_selected[0].upper()))
    print("TEAM: " + str(coach_selected[1][0]).upper())
    print("COST: " + str(coach_selected[1][1]))
    print("")
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print("")
    print("PLAYERS")
    
if len(keep_players) > 0:
    for p in keep_players:
        print("PLAYER: " + p.upper())
        print("ROLE: " + str(players_filtered[p][1]))
        print("COST: " + str(players_filtered[p][2]))
        print("SCORE: " + str(players_filtered[p][3]))
        print("PREDICTION: " + str(players_filtered[p][4]))
        print("-------------------")

for player,role,cost,pred,pts in portfolio:
    total_cost += cost
    total_points += pts
    print("PLAYER: " + player.upper())
    print("ROLE: " + str(role))
    print("COST: " + str(cost))
    print("SCORE: " + str(pred))
    print("PREDICTION: " + str(pts))
    print("-------------------")
print("TOTAL COST: " + str(total_cost))
print("TOTAL SCORE: " + str(total_points))

COACH: 
NAME: BRETT BROWN
TEAM: PHILADELPHIA 76ERS
COST: 8.5

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

PLAYERS
PLAYER: BEN SIMMONS
ROLE: G
COST: 16.1
SCORE: 26.981259842519684
PREDICTION: 46.94
-------------------
PLAYER: JOEL EMBIID
ROLE: C
COST: 15.8
SCORE: 26.58375
PREDICTION: 47.26
-------------------
PLAYER: AL HORFORD
ROLE: C
COST: 13.9
SCORE: 22.19625
PREDICTION: 39.46
-------------------
PLAYER: JAYSON TATUM
ROLE: A
COST: 9.9
SCORE: 15.244160583941605
PREDICTION: 33.15
-------------------
PLAYER: ROBERT COVINGTON
ROLE: A
COST: 8.8
SCORE: 13.828148148148147
PREDICTION: 28.72
-------------------
PLAYER: MARCUS MORRIS
ROLE: A
COST: 8.0
SCORE: 13.384887218045113
PREDICTION: 26.57
-------------------
PLAYER: MARCUS SMART
ROLE: G
COST: 7.8
SCORE: 13.119999999999997
PREDICTION: 27.88
-------------------
PLAYER: MARCO BELINELLI
ROLE: G
COST: 7.6
SCORE: 8.01986013986014
PREDICTION: 20.12
-------------------
PLAYER: SEMI OJELEYE
ROLE: A
COST: 4.1
SCORE: 1.6666666666666665
PREDICTION: 15.0
---